In [ ]:
import pandas as pd
# Modulos do Plotly
from plotly.subplots import make_subplots
import plotly.express as px

# Avisos
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Armazenar a informação do link publico do Google Shhet
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQariqhJpZEwWvUnakVKUnYqAFnLnOYllyvEw4MJFammqXlR08DHTLKGb7pwtglqFe7f2os3hqVwbmF/pubhtml'
# Realiza a Leitura do link Publico do Google Sheet
df = pd.read_html(url, header =0, index_col=0, skiprows=1)[0].reset_index()

In [24]:
# Realiza a extração apenas das 15 primeiras Colunas
df = df.iloc[:, :15]
# Apagar os Registros nulos da coluna "CLIENTE"
df = df.dropna(subset=['CLIENTE'])

In [25]:
# Cria uma função para Remover cifrão, trocar , por . para depois realizar trasformação do número para Float
def Tratar_Valor (x):
    x = x.replace('R$ ','')
    x = x.replace(',','.')
    x = x.replace(';','.')
    x = pd.to_numeric(x)
    return x

df['VALOR'] = df['VALOR'].apply(Tratar_Valor)

In [26]:
# Realiza a transformação da Coluna Data em formato de Data para se Extrair apenas o dia dessa Coluna
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')
df['DIA'] =  df['DATA'].dt.day
df['MES'] =  df['DATA'].dt.month

In [27]:
# Definir o mês atual
#mes_atual = datetime.now().month

# Calcular o mês anterior
#mes_anterior = mes_atual - 1 if mes_atual > 1 else 12

# Filtrar os dados do mês anterior
#df = df[df['DATA'].dt.month == mes_anterior]

# Filtrar os dados do mês atual
#df = df[df['data'].dt.month == mes_atual]

In [28]:
# Função para torna todos os Registros da Coluna UF em Maisculo
def MAISCULA(d):
    if isinstance(d, str):
        return d.upper()
    else:
        return d
df['UF'] = df['UF'].apply(MAISCULA)

In [29]:
# Realiza agrupamento da Tabela para criar a Primeira tabela de referencia para construção do Gráfico de Barras
DASH01 = df.groupby(by=['Vendedor']).agg(    VENDAS = ('VALOR','sum')).sort_values(by='VENDAS', ascending=True).reset_index()

In [30]:
# Realiza o agrupamento da Tabela para criar a Segunda tabela de referncia para contrusção do Gráfico de Rosca
DASH02 = df.groupby(by=['UF']).agg(  VENDAS = ('CLIENTE','count')).reset_index()
DASH02['%VENDAS'] = DASH02['VENDAS']/DASH02['VENDAS'].sum() * 100

In [31]:
# Realiza o agrupamento da Tabela para criar a Terceira tabela de referncia para contrusção do Gráfico de Linhas
DASH03 = df.groupby(by=['DIA']).agg( VENDAS = ('CLIENTE','count')).reset_index()

In [32]:
CARTAO = df.groupby(by=['CPF']).agg( TOTAL = ('CLIENTE','count')).reset_index().rename(columns={'CPF':'CATEGORIA'})

In [33]:
CARTAO_02 = df.groupby(by=['CPF']).agg( TOTAL = ('VALOR','mean')).reset_index().rename(columns={'CPF':'CATEGORIA'})
CARTAO_02['TOTAL'] = round(CARTAO_02['TOTAL'],2)

In [34]:
CARTAO_03 = df.groupby(by=['CPF']).agg( TOTAL = ('VALOR','sum')).reset_index().rename(columns={'CPF':'CATEGORIA'})
CARTAO_03['TOTAL'] = round(CARTAO_03['TOTAL'],0)

In [35]:
GRAFICO01 = px.pie(
    CARTAO,
    names='CATEGORIA', 
    values='TOTAL',
    #title='% Vendas Por Estado',
    #hole=0.6,
    color_discrete_sequence=px.colors.sequential.Cividis,
    template='plotly_dark',  # Escolha o modelo de fundo (opcional)
)

GRAFICO01.update_traces(
    textinfo='value', textfont_size=100,
    textposition='inside',
    marker=dict( line=dict(color='#000000', width=2) ) 
    )

GRAFICO01.update( layout_showlegend=False )

GRAFICO01

In [36]:
GRAFICO02 = px.pie(
    CARTAO_02,
    names='CATEGORIA', 
    values='TOTAL',
    #title='% Vendas Por Estado',
    #hole=0.6,
    color_discrete_sequence=px.colors.sequential.Cividis,
    template='plotly_dark',  # Escolha o modelo de fundo (opcional)
)

GRAFICO02.update_traces(
    textinfo='value', textfont_size=100,
    textposition='inside',
    marker=dict( line=dict(color='#000000', width=2) ) 
    )

GRAFICO02.update( layout_showlegend=False )

GRAFICO02

In [37]:
GRAFICO03 = px.pie(
    CARTAO_03,
    names='CATEGORIA', 
    values='TOTAL',
    #title='% Vendas Por Estado',
    #hole=0.6,
    color_discrete_sequence=px.colors.sequential.Cividis,
    template='plotly_dark',  # Escolha o modelo de fundo (opcional)
)

GRAFICO03.update_traces(
    textinfo='value', textfont_size=100,
    textposition='inside',
    marker=dict( line=dict(color='#000000', width=2)) 
    )



GRAFICO03.update( layout_showlegend=False )

GRAFICO03.update_traces()

In [38]:
# Gráfico de Barras
Figura_Barras = px.bar( 
                       DASH01, 
                       x='VENDAS', 
                       y='Vendedor', 
                       #title='Total de Vendas', 
                       text='VENDAS',
                       #template='plotly_dark',  # Escolha o modelo de fundo (opcional)
                       )

# Definindo o formato dos valores ao pairar o mouse sobre os pontos do gráfico
Figura_Barras.update_traces(
    textfont=dict(size=16, color='white', family='Arial'),
    texttemplate='R$ %{x:.2f}',
    )

# Ajustando a cor de fundo do gráfico
Figura_Barras.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',  # Define a cor da área de plotagem como transparente
    xaxis_showgrid=False)

Figura_Barras.show()

In [39]:
TOTAL = DASH02['VENDAS'].sum()
Figura_Pizza = px.pie(
    DASH02,
    names='UF', values='VENDAS',
    #title='% Vendas Por Estado',
    hole=0.6,
    #color_discrete_sequence=px.colors.sequential.Cividis,
    #template='plotly_dark',  # Escolha o modelo de fundo (opcional)
)

Figura_Pizza.update_traces(
    hoverinfo='label+percent', textinfo='percent+label', textfont_size=14,
    textposition='inside',
    marker=dict( line=dict(color='#ffffff', width=2) ) )

Figura_Pizza.add_annotation(
    x=0.5, y=0.5,
    showarrow=False,
    text=f'{ round(TOTAL)}',
    font=dict(
      #family='Courier New, monospace',
      size=30,
      color='#000000'
    ),
)

Figura_Pizza.update( layout_showlegend=True )

Figura_Pizza

In [40]:
# Criar o gráfico de linhas
Figura_Linhas = px.line(
    DASH03, x='DIA', 
    y='VENDAS', 
    #title='Vendas Por Dia', 
    labels={'DIA': 'DIAS', 'VENDAS': 'Vendas'}, 
    line_shape='linear', 
    text='VENDAS',
    #template='plotly_dark',  # Escolha o modelo de fundo (opcional)
)

# Definindo o formato dos valores ao pairar o mouse sobre os pontos do gráfico
Figura_Linhas.update_traces(
    #mode='lines+markers',
    textposition='top center',
    fill='tozeroy',
    fillcolor='rgba(198, 213, 245, 0.5)',
    textfont=dict(size=13, color='blue', family='Arial'),
    showlegend=False  # Não mostrar a legenda
)

# Ajustando a cor de fundo do gráfico
Figura_Linhas.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',  # Define a cor da área de plotagem como transparente
    yaxis_range=[0, DASH03.VENDAS.max() + ( DASH03.VENDAS.max() * 0.25 ) ],
    xaxis=dict(type='category', categoryorder='array', categoryarray=DASH03['DIA']),  # Define o eixo X como categórico
    xaxis_showgrid=False,
)

# Adiciona pontos nos dias sem dados
for dia in DASH03['DIA']:
    if dia not in Figura_Linhas.data[0]['x']:
        Figura_Linhas.add_trace(go.Scatter(x=[dia], y=[0], mode='markers', marker=dict(size=10, color='red')))

# Exibir a figura
Figura_Linhas.show()


In [41]:
# Criar um grid de gráficos
Figura = make_subplots(
    # Layout do relatorio
    rows=7, cols=6,
    # Design
    specs=[
      # 1 linha
      [ { 'colspan' : 2, 'rowspan' : 1, 'type':'pie' }, None, { 'colspan' : 2, 'rowspan' : 1, 'type':'pie' }, None, { 'colspan' : 2, 'rowspan' : 1, 'type':'pie' }, None ],
      # 2 linha
      [ { 'colspan' : 6, 'rowspan' : 3 }, None, None, None, None, None ],
      # 3 linha
      [ None, None, None, None, None, None ],
      # 4 linha
      [ None, None,None, None,  None, None ],
      # 5 linha
      [ { 'colspan': 3, 'rowspan' : 3 }, None, None ,{ 'colspan' : 3, 'rowspan' : 3, 'type':'pie'}, None, None ],
      # 6 linha
      [ None, None, None, None, None, None ],
      # 7 linha
      [ None, None, None, None, None, None ]
    ],
    # Titulos
    #subplot_titles=( [Loop for Loop in range(1, 21)] )
    subplot_titles=('Total Vendas', 'Média de Vendas', 'Valor de Vendas', 'Vendas por Vendendor', 'Quantidade de Vendas por Dia', '%Vendas por Estado' ), 
)

In [42]:
# Adicionado os gráficos no relatorio
Figura.add_trace(GRAFICO01.data[0], row=1, col=1 )
Figura.add_trace(GRAFICO02.data[0], row=1, col=3 )
Figura.add_trace(GRAFICO03.data[0], row=1, col=5 )
Figura.add_trace(Figura_Linhas.data[0], row=2, col=1 )
Figura.add_trace(Figura_Barras.data[0], row=5, col=1 )
Figura.add_trace(Figura_Pizza.data[0], row=5, col=4 )

# Criar subplots
### ### ### ### ### ###
### Customatização ###
### ### ### ### ### ###

# Ajuste na Figura
Figura.update_layout(
    title_font_size=30,
    # Tamanhos
    width=1200, height=1000,
    plot_bgcolor='rgba(0,0,0,0)',  # Define a cor da área de plotagem como transparente
    # Titulo geral
    title=dict(text='Dashboard - De Vendas', x=0.5)),
    

Figura.update_traces(
    # Desativas Legendas
    showlegend=False,
)

Figura